In [11]:
# Tự động reload lại code khi bạn sửa trong file .py
%load_ext autoreload
%autoreload 2

import time
import pandas as pd # Dùng pandas hiển thị bảng lỗi cho đẹp

# Import các hàm từ file .py của bạn
from utils.data_loader import load_conll_data, build_dictionary
from utils.metrics import count_correct_words, calculate_metrics
from methods.baseline import LongestMatching

# Cấu hình đường dẫn file (Check lại tên file của bạn)
TRAIN_PATH = 'data/train.conll'
TEST_PATH = 'data/test.conll'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
train_sentences = load_conll_data(TRAIN_PATH)
test_sentences = load_conll_data(TEST_PATH)

print(f"Train size: {len(train_sentences)} câu")
print(f"Test size:  {len(test_sentences)} câu")

# Xây dựng từ điển 
vocab = build_dictionary(train_sentences)
print(f"Kích thước từ điển: {len(vocab)} từ")

Train size: 8977 câu
Test size:  1020 câu
Kích thước từ điển: 12756 từ


Longest Matching test

In [9]:
# Khởi tạo mô hình
lm_model = LongestMatching(vocab)

start_time = time.time()

predicted_sentences = []
# Chạy dự đoán
for gold_sentence in test_sentences:
    input_text = " ".join(gold_sentence).replace("_", " ")
    
    pred = lm_model.segment(input_text)
    predicted_sentences.append(pred)

end_time = time.time()
print(f"Hoàn thành trong {end_time - start_time:.2f} giây.")

Hoàn thành trong 0.05 giây.


Tính điểm

In [12]:
# Tính điểm
correct, total_pred, total_gold = count_correct_words(predicted_sentences, test_sentences)
p, r, f1 = calculate_metrics(correct, total_pred, total_gold)

print("KẾT QUẢ: ")
print(f"Precision: {p*100:.2f}%")
print(f"Recall:    {r*100:.2f}%")
print(f"F1-Score:  {f1*100:.2f}%")

KẾT QUẢ: 
Precision: 94.09%
Recall:    95.21%
F1-Score:  94.64%


In [14]:
# Tìm các câu làm sai
errors = []
for i in range(len(test_sentences)):
    if predicted_sentences[i] != test_sentences[i]:
        errors.append({
            "Input": " ".join(test_sentences[i]).replace("_", " "),
            "Đúng": " ".join(test_sentences[i]),
            "Dự đoán": " ".join(predicted_sentences[i])
        })
        
    if len(errors) >= 10: break 

df_errors = pd.DataFrame(errors)

print("DANH SÁCH CÁC CÂU BỊ TÁCH SAI:")
display(df_errors)

🔍 DANH SÁCH CÁC CÂU BỊ TÁCH SAI:


,Input,Đúng,Dự đoán
0,Đây là Phan Rang .,Đây là Phan_Rang .,Đây là Phan Rang .
1,Mất uy tín với đồng bọn ...,Mất uy_tín với đồng_bọn ...,Mất uy_tín với đồng bọn ...
2,Đi từ khó nghèo ...,Đi từ khó nghèo ...,Đi từ khó_nghèo ...
3,VKSND tỉnh Bến Tre kháng nghị .,VKSND tỉnh Bến_Tre kháng_nghị .,VKSND tỉnh Bến_Tre kháng nghị .
4,Cả vườn thú buồn thương .,Cả vườn thú buồn thương .,Cả vườn_thú buồn thương .
5,Rồi chúng tôi lại ra đi .,Rồi chúng_tôi lại ra đi .,Rồi chúng_tôi lại ra_đi .
6,Cuộc thi Ký sự nhân vật 2005 .,Cuộc thi Ký_sự nhân_vật 2005 .,Cuộc_thi Ký_sự nhân_vật 2005 .
7,Trại Kem Kjang ... 864 giờ khắc khoải .,Trại Kem_Kjang ... 864 giờ khắc_khoải .,Trại Kem Kjang ... 864 giờ_khắc khoải .
8,Cứ thế đã gần 20 năm .,Cứ thế đã gần 20 năm .,Cứ_thế đã gần 20 năm .
9,Chị ấy rất độ lượng và thông cảm .,Chị ấy rất độ_lượng và thông_cảm .,Chị ấy rất độ lượng và thông_cảm .


In [37]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

# 1. Hàm rút trích đặc trưng (Feature Extraction)
# CRF cần nhìn vào "ngữ cảnh" xung quanh từ để quyết định
def word2features(sent, i):
    word = sent[i][0] # Lấy từ (syllable)
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(), # Có viết hoa không?
        'word.istitle()': word.istitle(), # Viết hoa chữ đầu?
        'word.isdigit()': word.isdigit(), # Là số?
    }
    
    # Nhìn từ phía trước (Previous word)
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
        })
    else:
        features['BOS'] = True # Đầu câu

    # Nhìn từ phía sau (Next word)
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
        })
    else:
        features['EOS'] = True # Cuối câu

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

# 2. Chuyển đổi dữ liệu sang dạng B-I-W (Begin - Inside - Word)
# Vì CRF là bài toán gán nhãn, ta cần chuyển tách từ về gán nhãn
# Ví dụ: "Học_sinh" -> [("Học", "B_W"), ("sinh", "I_W")]
def convert_to_bi_labels(sentences):
    formatted_data = []
    for sent in sentences:
        labeled_sent = []
        for word in sent:
            syllables = word.split('_')
            # Âm tiết đầu tiên của từ gán nhãn B_W
            labeled_sent.append((syllables[0], 'B_W'))
            # Các âm tiết sau gán nhãn I_W
            for s in syllables[1:]:
                labeled_sent.append((s, 'I_W'))
        formatted_data.append(labeled_sent)
    return formatted_data

# --- CHẠY THỰC NGHIỆM ---

# A. Chuyển đổi dữ liệu
print("Đang chuyển đổi dữ liệu sang format B-I-W...")
train_data_bi = convert_to_bi_labels(train_sentences)
test_data_bi = convert_to_bi_labels(test_sentences)

# B. Tạo Features (X) và Labels (y)
X_train = [sent2features(s) for s in train_data_bi]
y_train = [sent2labels(s) for s in train_data_bi]

X_test = [sent2features(s) for s in test_data_bi]
y_test = [sent2labels(s) for s in test_data_bi]

# C. Huấn luyện mô hình (Training)
print(" Đang huấn luyện mô hình CRF")
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.046,
    c2=0.049,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

# D. Dự đoán và Đánh giá
print(" -> Đang dự đoán trên tập Test...")
y_pred = crf.predict(X_test)

# Hàm chuyển đổi ngược từ nhãn B-I về câu tách từ để tính F1 cũ
def rebuild_sentences(tokens, labels):
    rebuilt_sentences = []
    for i in range(len(tokens)):
        sent_tokens = [t[0] for t in tokens[i]] # Lấy danh sách âm tiết
        sent_labels = labels[i]
        
        words = []
        current_word = []
        for j, (token, label) in enumerate(zip(sent_tokens, sent_labels)):
            if label == 'B_W':
                if current_word:
                    words.append("_".join(current_word))
                current_word = [token]
            elif label == 'I_W':
                current_word.append(token)
            else: # Trường hợp nhãn lạ, coi như từ mới
                if current_word:
                    words.append("_".join(current_word))
                current_word = [token]
        if current_word:
            words.append("_".join(current_word))
        rebuilt_sentences.append(words)
    return rebuilt_sentences

# Tái tạo lại câu từ kết quả dự đoán của CRF
pred_sentences_crf = rebuild_sentences(test_data_bi, y_pred)

# Tính điểm bằng hàm cũ của bạn
c, tp, tg = count_correct_words(pred_sentences_crf, test_sentences)
p, r, f1 = calculate_metrics(c, tp, tg)

print("\n" + "="*40)
print(" KẾT QUẢ CẢI TIẾN (CRF MODEL)")
print("="*40)
print(f" Precision: {p*100:.2f}%")
print(f" Recall:    {r*100:.2f}%")
print(f" F1-Score:  {f1*100:.2f}%")
print("="*40)

Đang chuyển đổi dữ liệu sang format B-I-W...
 Đang huấn luyện mô hình CRF
 -> Đang dự đoán trên tập Test...

 KẾT QUẢ CẢI TIẾN (CRF MODEL)
 Precision: 91.00%
 Recall:    91.51%
 F1-Score:  91.26%


In [33]:

from methods.CFR import CRFSegmenter # Import từ file mới tạo
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.0406,
    c2=0.0494,
    max_iterations=100,
    all_possible_transitions=True
)
print("🚀BẮT ĐẦU THỰC NGHIỆM CẢI TIẾN (CRF MODEL)")

# 1. Khởi tạo và Train mô hình
crf_model = CRFSegmenter(dictionary=vocab) 

crf_model.train(train_sentences) # Dùng lại biến train_sentences đã load ở trên

# 2. Chạy thử nghiệm trên tập Test
print("\n Đang chạy dự đoán trên tập Test...")
crf_predictions = []

import time
start = time.time()
for gold in test_sentences:
    raw_input = " ".join(gold).replace("_", " ")
    pred = crf_model.segment(raw_input) # Gọi hàm segment y hệt Baseline
    crf_predictions.append(pred)
end = time.time()
print(f" Hoàn thành trong {end - start:.2f} giây.")

# 3. Đánh giá kết quả
c, tp, tg = count_correct_words(crf_predictions, test_sentences)
p, r, f1 = calculate_metrics(c, tp, tg)

print("\n" + "="*40)
print("KẾT QUẢ MODEL CẢI TIẾN (CRF)")
print("="*40)
print(f"Precision: {p*100:.2f}%")
print(f"Recall:    {r*100:.2f}%")
print(f" F1-Score:  {f1*100:.2f}%")
print("="*40)

🚀BẮT ĐẦU THỰC NGHIỆM CẢI TIẾN (CRF MODEL)
 -> [CRF Hybrid] Đang chuẩn bị dữ liệu & features...
 -> [CRF Hybrid] Đang training model...
 -> [CRF Hybrid] Training hoàn tất!

 Đang chạy dự đoán trên tập Test...
 Hoàn thành trong 0.15 giây.

KẾT QUẢ MODEL CẢI TIẾN (CRF)
Precision: 91.54%
Recall:    92.09%
 F1-Score:  91.81%


In [34]:
import methods.CFR
import importlib

# Ép tải lại file crf_model.py mới nhất từ đĩa cứng
importlib.reload(methods.CFR)

# Import lại class sau khi reload
from methods.CFR import CRFSegmenter

print(" Đã cập nhật code mới thành công!")

 Đã cập nhật code mới thành công!


In [38]:

import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn_crfsuite import metrics
from methods.CFR import CRFSegmenter # Import class của bạn

print(" ĐANG CHUẨN BỊ DỮ LIỆU ĐỂ TUNING...")

segmenter = CRFSegmenter(dictionary=vocab)

print(" -> Đang tạo Features cho tập Train...")
train_data_bi = segmenter._convert_to_bi_labels(train_sentences)
X_train = [segmenter._sent2features(s) for s in train_data_bi]
y_train = [segmenter._sent2labels(s) for s in train_data_bi]

# 3. Định nghĩa không gian tham số cần thử
# c1 và c2 sẽ được lấy ngẫu nhiên theo phân phối mũ (Exponential distribution)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# 4. Thiết lập mô hình CRF cơ bản
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)

# Sử dụng F1-Score trung bình để đánh giá
f1_scorer = make_scorer(metrics.flat_f1_score, average='weighted', labels=['B_W', 'I_W'])

rs = RandomizedSearchCV(
    crf, 
    params_space, 
    cv=3, 
    verbose=1, 
    n_jobs=-1, 
    n_iter=10, 
    scoring=f1_scorer
)

print(f"\nBẮT ĐẦU CHẠY TÌM KIẾM (Sẽ thử {10 * 3} lượt train)...")
try:
    rs.fit(X_train, y_train)
    
    # 6. In kết quả tốt nhất
    print("\n" + "="*40)
    print("KẾT QUẢ TỐI ƯU (BEST PARAMS)")
    print("="*40)
    print('Best params:', rs.best_params_)
    print('Best CV score:', rs.best_score_)
    print("="*40)

    # 7. Lưu lại tham số tốt nhất để dùng
    best_c1 = rs.best_params_['c1']
    best_c2 = rs.best_params_['c2']
    
except Exception as e:
    print("\nCó lỗi xảy ra:", e)
    # Nếu lỗi, dùng tham số mặc định
    best_c1 = 0.1
    best_c2 = 0.1

print(f"\n c1={best_c1:.4f}, c2={best_c2:.4f}")

 ĐANG CHUẨN BỊ DỮ LIỆU ĐỂ TUNING...
 -> Đang tạo Features cho tập Train...

BẮT ĐẦU CHẠY TÌM KIẾM (Sẽ thử 30 lượt train)...
Fitting 3 folds for each of 10 candidates, totalling 30 fits

KẾT QUẢ TỐI ƯU (BEST PARAMS)
Best params: {'c1': np.float64(0.33995133103188707), 'c2': np.float64(0.017641623395822585)}
Best CV score: 0.950281044833933

 c1=0.3400, c2=0.0176
